In [7]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [2]:
!pip install openai pandas


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import openai
import pandas as pd

# 配置 API 密钥
openai.api_key = 'sk-dddfebca99f24567ba0742500e1f3fc0'
openai.api_base = 'https://api.deepseek.com/v1'


In [5]:
# 读取 TSV 文件
input_file = '/content/gdrive/MyDrive/Subtask_A_Train/Subtask A train /subtask_a_train.tsv'  # 替换为您的 TSV 文件路径
data = pd.read_csv(input_file, sep='\t')

# 检查是否包含必要的列
if 'compound' not in data.columns or 'sentence' not in data.columns:
    raise ValueError("TSV 文件必须包含 'compound' 和 'sentence' 列。")


In [6]:
def classify_sentence_type(compound, sentence):
    prompt = f"""
    You are an idiom expert. Your task is to analyze the given compound word and sentence. Determine whether the compound word is used idiomatically or literally in the given context. Respond with either "idiomatic" or "literal" based on your analysis.

    Compound: {compound}
    Sentence: {sentence}

    Output:
    """
    try:
        response = openai.ChatCompletion.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are an idiom analysis expert."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"


In [7]:
# 遍历每一行并打印结果
for _, row in data.iterrows():
    compound = row['compound']
    sentence = row['sentence']
    sentence_type = classify_sentence_type(compound, sentence)
    print(f"Compound：{compound}, Sentence Type：{sentence_type}")


Compound：elbow grease, Sentence Type：Output: idiomatic  

Explanation: In this context, "elbow grease" is used idiomatically to mean hard physical effort or manual labor, rather than literally referring to grease applied to an elbow. The sentence implies that significant effort was required to restore the engine, not that actual grease was used.
Compound：night owl, Sentence Type：Output: idiomatic

Explanation: In the sentence, "night owl" is used idiomatically to describe someone who stays up late at night and is more active during the evening or nighttime hours. It does not refer to an actual owl or a literal bird, but rather to a person's habits and preferences.
Compound：heart of gold, Sentence Type：idiomatic  

Explanation: The phrase "heart of gold" is used idiomatically in this context. It does not literally mean that the person has a heart made of gold, but rather figuratively describes someone who is kind, generous, and compassionate. The sentence emphasizes that despite the pri

In [10]:
def classify_sentence_type(compound, tag):
    prompt = f"""
    You are an idiom expert. Your task is to give the idiomatic or literal meaning of the compound words, according to the given tag ('idiomatic' or 'literal').
    The meanings should not exceed 30 words.

    Compound: {compound}
    Tag: {tag}

    Output:
    - Meaning: [brief explanation of the compound word, maximum 30 words]
    """
    try:
        response = openai.ChatCompletion.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are an idiom analysis expert."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

# 遍历每一行并打印结果
for _, row in data.iterrows():
    compound = row['compound']
    sentence_type = row['sentence_type']  # 假设 TSV 文件中有 'sentence_type' 列
    meaning = classify_sentence_type(compound, sentence_type)
    print(f"Compound: {compound}, Sentence Type: {sentence_type}, Meaning: {meaning}")


Compound: elbow grease, Sentence Type: idiomatic, Meaning: - Meaning: Hard physical effort or manual labor, often required to complete a task or clean something thoroughly.
Compound: night owl, Sentence Type: idiomatic, Meaning: - Meaning: A person who stays up late at night or is active during nighttime.
Compound: heart of gold, Sentence Type: idiomatic, Meaning: - Meaning: A kind, generous, and compassionate nature.
Compound: agony aunt, Sentence Type: idiomatic, Meaning: - Meaning: A person, often a columnist, who offers advice and sympathy to people experiencing personal problems.
Compound: shrinking violet, Sentence Type: idiomatic, Meaning: - Meaning: A shy, timid, or introverted person who avoids attention or social interaction. (idiomatic)
Compound: green fingers, Sentence Type: literal, Meaning: - Meaning: Fingers that are physically green in color, possibly due to staining from plants, paint, or other substances. (Literal interpretation)
Compound: ancient history, Sentence Ty